In [205]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [165]:
train_path = "H:\\Hobby\\ML Dataset\\titanic\\train.csv"
test_path = "H:\\Hobby\\ML Dataset\\titanic\\test.csv"

In [166]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)


y=train['Survived']
train=train.drop(['Survived','PassengerId', 'Ticket', 'Cabin'], axis=1)
passID = test['PassengerId']
test = test.drop(['PassengerId', 'Ticket', 'Cabin'], axis=1)

In [167]:
train_test_data = [train, test] 

for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

title_mapping = {"Mr": "Mr","Mrs": "Mrs", "Miss": "Miss" ,"Master": 'O', "Dr": 'O', "Rev": 'O', "Col": 'O', "Major": 'O', "Mlle": 'O',"Countess": 'O',
                 "Ms": 'O', "Lady": 'O', "Jonkheer": 'O', "Don": 'O', "Dona" : 'O', "Mme": 'O',"Capt": 'O',"Sir": 'O' }

for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset.drop(['Name'], axis=1, inplace=True)

In [168]:
train['Embarked'] = train['Embarked'].fillna('S')

embark = pd.get_dummies(train['Embarked'], drop_first=True)
sex = pd.get_dummies(train['Sex'], drop_first=True)
title = pd.get_dummies(train['Title'], drop_first=True)
train = pd.concat([train, sex, embark, title], sort=False, axis=1)
train.drop(['Embarked', 'Sex', 'Title'], axis=1, inplace=True)

embark = pd.get_dummies(test['Embarked'], drop_first=True)
sex = pd.get_dummies(test['Sex'], drop_first=True)
title = pd.get_dummies(test['Title'], drop_first=True)
test = pd.concat([test, sex, embark, title], sort=False, axis=1)
test.drop(['Embarked', 'Sex', 'Title'], axis=1, inplace=True)

In [169]:
train.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'male', 'Q', 'S', 'Mr',
       'Mrs', 'O'],
      dtype='object')

In [170]:
train[train]

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'male', 'Q', 'S', 'Mr',
       'Mrs', 'O'],
      dtype='object')

In [265]:
# encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
# OH_train = pd.DataFrame(encoder.fit_transform(train[cols]))
# OH_test = pd.DataFrame(encoder.transform(test[cols]))

In [266]:
# OH_train.index = train.index
# OH_test.index = test.index

In [267]:
# train = train.drop(cols, axis=1)
# test = test.drop(cols, axis=1)

# train = pd.concat([train, OH_train], axis=1)
# test = pd.concat([test, OH_test], axis=1)

In [163]:
train.columns

RangeIndex(start=0, stop=11, step=1)

In [171]:
imputer = SimpleImputer()

temp = test.copy()


train = pd.DataFrame(imputer.fit_transform(train))
test = pd.DataFrame(imputer.transform(test))

train.columns = temp.columns
test.columns = temp.columns

In [173]:
test.isnull().sum()

Pclass    0
Age       0
SibSp     0
Parch     0
Fare      0
male      0
Q         0
S         0
Mr        0
Mrs       0
O         0
dtype: int64

In [193]:
xTrain, xTest, yTrain, yTest = train_test_split(train, y, random_state=101, test_size=0.3)

model = DecisionTreeClassifier()
model.fit(xTrain, yTrain)
preds = model.predict(xTest)

print(accuracy_score(preds, yTest))
confusion_matrix(preds, yTest)


0.7873134328358209


array([[133,  36],
       [ 21,  78]], dtype=int64)

In [235]:
model = RandomForestClassifier(n_estimators=50, max_leaf_nodes=20)
model.fit(xTrain, yTrain)
preds = model.predict(xTest)
print(accuracy_score(preds, yTest))
confusion_matrix(preds, yTest)

0.8208955223880597


array([[141,  35],
       [ 13,  79]], dtype=int64)

In [215]:
param_grid = {'n_estimators': [50,100,150,200,300,400,500,1000],
             'max_leaf_nodes': [20,50,100,150]}

model = GridSearchCV(RandomForestClassifier(), param_grid, verbose=3)
model.fit(xTrain, yTrain)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] max_leaf_nodes=20, n_estimators=50 ..............................
[CV] .. max_leaf_nodes=20, n_estimators=50, score=0.840, total=   0.2s

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



[CV] max_leaf_nodes=20, n_estimators=50 ..............................
[CV] .. max_leaf_nodes=20, n_estimators=50, score=0.856, total=   0.2s
[CV] max_leaf_nodes=20, n_estimators=50 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s


[CV] .. max_leaf_nodes=20, n_estimators=50, score=0.760, total=   0.2s
[CV] max_leaf_nodes=20, n_estimators=50 ..............................
[CV] .. max_leaf_nodes=20, n_estimators=50, score=0.839, total=   0.2s
[CV] max_leaf_nodes=20, n_estimators=50 ..............................
[CV] .. max_leaf_nodes=20, n_estimators=50, score=0.831, total=   0.2s
[CV] max_leaf_nodes=20, n_estimators=100 .............................
[CV] . max_leaf_nodes=20, n_estimators=100, score=0.832, total=   0.4s
[CV] max_leaf_nodes=20, n_estimators=100 .............................
[CV] . max_leaf_nodes=20, n_estimators=100, score=0.840, total=   0.4s
[CV] max_leaf_nodes=20, n_estimators=100 .............................
[CV] . max_leaf_nodes=20, n_estimators=100, score=0.752, total=   0.4s
[CV] max_leaf_nodes=20, n_estimators=100 .............................
[CV] . max_leaf_nodes=20, n_estimators=100, score=0.839, total=   0.4s
[CV] max_leaf_nodes=20, n_estimators=100 .............................
[CV] .

[CV] . max_leaf_nodes=50, n_estimators=300, score=0.808, total=   1.1s
[CV] max_leaf_nodes=50, n_estimators=300 .............................
[CV] . max_leaf_nodes=50, n_estimators=300, score=0.816, total=   1.1s
[CV] max_leaf_nodes=50, n_estimators=300 .............................
[CV] . max_leaf_nodes=50, n_estimators=300, score=0.752, total=   1.2s
[CV] max_leaf_nodes=50, n_estimators=300 .............................
[CV] . max_leaf_nodes=50, n_estimators=300, score=0.839, total=   1.1s
[CV] max_leaf_nodes=50, n_estimators=300 .............................
[CV] . max_leaf_nodes=50, n_estimators=300, score=0.798, total=   1.1s
[CV] max_leaf_nodes=50, n_estimators=400 .............................
[CV] . max_leaf_nodes=50, n_estimators=400, score=0.808, total=   1.5s
[CV] max_leaf_nodes=50, n_estimators=400 .............................
[CV] . max_leaf_nodes=50, n_estimators=400, score=0.856, total=   1.4s
[CV] max_leaf_nodes=50, n_estimators=400 .............................
[CV] .

[CV]  max_leaf_nodes=100, n_estimators=1000, score=0.815, total=   3.7s
[CV] max_leaf_nodes=100, n_estimators=1000 ...........................
[CV]  max_leaf_nodes=100, n_estimators=1000, score=0.806, total=   4.0s
[CV] max_leaf_nodes=150, n_estimators=50 .............................
[CV] . max_leaf_nodes=150, n_estimators=50, score=0.792, total=   0.2s
[CV] max_leaf_nodes=150, n_estimators=50 .............................
[CV] . max_leaf_nodes=150, n_estimators=50, score=0.832, total=   0.3s
[CV] max_leaf_nodes=150, n_estimators=50 .............................
[CV] . max_leaf_nodes=150, n_estimators=50, score=0.744, total=   0.3s
[CV] max_leaf_nodes=150, n_estimators=50 .............................
[CV] . max_leaf_nodes=150, n_estimators=50, score=0.798, total=   0.2s
[CV] max_leaf_nodes=150, n_estimators=50 .............................
[CV] . max_leaf_nodes=150, n_estimators=50, score=0.823, total=   0.2s
[CV] max_leaf_nodes=150, n_estimators=100 ............................
[CV]

[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:  3.4min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              ra

In [216]:
model.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=20, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [211]:
model = SVC(C=100, gamma=0.0001)
model.fit(xTrain, yTrain)
preds = model.predict(xTest)
print(accuracy_score(preds, yTest))

confusion_matrix(preds, yTest)

0.7723880597014925


array([[129,  36],
       [ 25,  78]], dtype=int64)

In [208]:
param_grid = {'C': [0.001,0.01,0.1,1,10,100],
             'gamma': [1,0.1,0.01,0.001,0.0001,0.00001]}

model = GridSearchCV(SVC(), param_grid, verbose=3)
model.fit(xTrain, yTrain)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] C=0.001, gamma=1 ................................................
[CV] .................... C=0.001, gamma=1, score=0.632, total=   0.0s
[CV] C=0.001, gamma=1 ................................................
[CV] .................... C=0.001, gamma=1, score=0.632, total=   0.0s
[CV] C=0.001, gamma=1 ................................................
[CV] .................... C=0.001, gamma=1, score=0.632, total=   0.0s
[CV] C=0.001, gamma=1 ................................................
[CV] .................... C=0.001, gamma=1, score=0.637, total=   0.0s
[CV] C=0.001, gamma=1 ................................................
[CV] .................... C=0.001, gamma=1, score=0.637, total=   0.0s

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s



[CV] C=0.001, gamma=0.1 ..............................................
[CV] .................. C=0.001, gamma=0.1, score=0.632, total=   0.0s
[CV] C=0.001, gamma=0.1 ..............................................
[CV] .................. C=0.001, gamma=0.1, score=0.632, total=   0.0s
[CV] C=0.001, gamma=0.1 ..............................................
[CV] .................. C=0.001, gamma=0.1, score=0.632, total=   0.0s
[CV] C=0.001, gamma=0.1 ..............................................
[CV] .................. C=0.001, gamma=0.1, score=0.637, total=   0.0s
[CV] C=0.001, gamma=0.1 ..............................................
[CV] .................. C=0.001, gamma=0.1, score=0.637, total=   0.0s
[CV] C=0.001, gamma=0.01 .............................................
[CV] ................. C=0.001, gamma=0.01, score=0.632, total=   0.0s
[CV] C=0.001, gamma=0.01 .............................................
[CV] ................. C=0.001, gamma=0.01, score=0.632, total=   0.0s
[CV] 

[CV] ...................... C=0.1, gamma=1, score=0.637, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] .................... C=0.1, gamma=0.1, score=0.632, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] .................... C=0.1, gamma=0.1, score=0.632, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] .................... C=0.1, gamma=0.1, score=0.632, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] .................... C=0.1, gamma=0.1, score=0.637, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] .................... C=0.1, gamma=0.1, score=0.637, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................... C=0.1, gamma=0.01, score=0.632, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] .

[CV] ....................... C=10, gamma=1, score=0.694, total=   0.0s
[CV] C=10, gamma=1 ...................................................
[CV] ....................... C=10, gamma=1, score=0.645, total=   0.0s
[CV] C=10, gamma=0.1 .................................................
[CV] ..................... C=10, gamma=0.1, score=0.712, total=   0.0s
[CV] C=10, gamma=0.1 .................................................
[CV] ..................... C=10, gamma=0.1, score=0.712, total=   0.0s
[CV] C=10, gamma=0.1 .................................................
[CV] ..................... C=10, gamma=0.1, score=0.680, total=   0.0s
[CV] C=10, gamma=0.1 .................................................
[CV] ..................... C=10, gamma=0.1, score=0.766, total=   0.0s
[CV] C=10, gamma=0.1 .................................................
[CV] ..................... C=10, gamma=0.1, score=0.758, total=   0.0s
[CV] C=10, gamma=0.01 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed:    6.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [210]:
model.best_estimator_


SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [227]:
model = RandomForestClassifier(n_estimators=50, max_leaf_nodes=20)
model.fit(train, y)
preds = model.predict(test)

output = pd.DataFrame({'PassengerId': passID,
                      'Survived': preds})

output.to_csv('6thSubmission.csv', index=False)